In [1]:
import pandas as pd
import json

In [2]:
def delete_colon(x):
    if "/" in x:
        return x.split("/")[-1]
    else:
        return x.split(":")[0] + x.split(":")[1]

In [47]:
def get_data(x,one_data,property_dict,i,j): #解析单个数据
    p = list(i.keys())[j]
    if("http://" in str(p)):
        p = p.split("/")[-1]
    relation = {}
    one_property = {}
    if("http://" in str(x)):
        #这里都是关系
        #这里还需要对其进行一个遍历
        relation['Start'] = one_data
        relation['End'] = x
        relation['Relationship'] = property_dict[p]
        
        return relation
        
    else:
        #这里都是属性
        #print(one_data)
        one_property["ID"] = one_data
        one_property['Property'] = property_dict[list(i.keys())[j]] 
        one_property['value'] = x
        
        return one_property

In [48]:
def get_entity(x): #获取所有实体信息
    entity_df = pd.DataFrame(columns = ['ID','name','label'])
    with open(x,encoding="utf-8") as f:
        a= json.load(f)

    class_dict={}
    property_dict={}
    for i in a['@graph']:
        if (i['@type'].split('#')[-1] =="Class"):
            class_dict[i['@id']] = i['label']['@value']
        elif (i['@type'].split('#')[-1] =="Property"):
            p_id = delete_colon(i['@id'])
            property_dict[p_id] = i['label']['@value']
        else:
            one_data={}
            one_data['ID'] = i['@id']
            one_data['name'] = i['label']['@value']
            one_data['label'] = class_dict[i["@type"]]
            if i['@id'] not in entity_df['ID']: 
                entity_df= entity_df.append(one_data,ignore_index=True)
                
    
    entity_df.columns = [':ID','name',':LABEL']
    return entity_df,property_dict
    


In [76]:
def get_rp(x,property_dict): #获取relationship和property
    relationship_df = pd.DataFrame(columns=['Start','End','Relationship'])
    property_df = pd.DataFrame(columns = ['ID','Property','value'])
    with open(x,encoding="utf-8") as f:
        a= json.load(f)
        
    print(a)
    for i in a['@graph']:
            #print(i)
           
            if "P" in str(i.keys()):
                
                for j in range(2,len(list(i.keys()))-1): 
                    if("P" in list(i.keys())[j]):
                        if(type(i[list(i.keys())[j]])==list):
                            for m in i[list(i.keys())[j]]:
                                if("http://" in str(m)):
                                    relationship_df = relationship_df.append(get_data(m,i['@id'],property_dict,i,j),ignore_index=True)
                                else:
                                    property_df = property_df.append(get_data(i[list(i.keys())[j]],one_data,property_dict,i,j),ignore_index=True)
                        else:
                            m = i[list(i.keys())[j]]

                            if("http://" in str(m)):
                                relationship_df = relationship_df.append(get_data(m,i['@id'],property_dict,i,j),ignore_index=True)
                            else:
                                property_df = property_df.append(get_data(i[list(i.keys())[j]],i['@id'],property_dict,i,j),ignore_index=True)
    relationship_df.columns=[':START',':END','relationship']
    return relationship_df,property_df

In [73]:
def get_nodes(entity_df, property_df):  #将三元组property和entity整合在一起成一个大表
    columns = list(property_df['Property'].unique())
    new_data = pd.DataFrame(columns= ([':ID','name',':LABEL'] + columns))
    new_data[[':ID','name',':LABEL']] = entity_df[[':ID','name',':LABEL']]
    new_data = new_data.set_index(':ID')
    
    property_df = property_df.set_index('ID')
    
    for i in property_df.index:
        new_data.loc[i][property_df.loc[i]['Property']] = property_df.loc[i]['value']
    return new_data

In [77]:
entity_df,property_dict = get_entity('character-covid-19-v0.2.json')
#entity_df_id = entity_df.set_index(":ID")
relationship_df,property_df = get_rp('character-covid-19-v0.2.json',property_dict)     

{'@graph': [{'@id': 'http://www.openkg.cn/COVID-19/character/class/C1', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '人物'}}, {'@id': 'http://www.openkg.cn/COVID-19/character/class/C10', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '城市'}}, {'@id': 'http://www.openkg.cn/COVID-19/character/class/C11', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '履历'}}, {'@id': 'http://www.openkg.cn/COVID-19/character/class/C12', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '职务'}}, {'@id': 'http://www.openkg.cn/COVID-19/character/class/C13', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '奖项'}}, {'@id': 'http://www.openkg.cn/COVID-19/character/class/C14', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '成果'}}, {'@id': 'ht

In [52]:
relationship_df

,:START,:END,relationship
0,http://www.openkg.cn/COVID-19/character/resour...,http://www.openkg.cn/COVID-19/character/resour...,导致
1,http://www.openkg.cn/COVID-19/character/resour...,http://www.openkg.cn/COVID-19/character/resour...,导致
2,http://www.openkg.cn/COVID-19/character/resour...,http://www.openkg.cn/COVID-19/character/resour...,所在城市
3,http://www.openkg.cn/COVID-19/character/resour...,http://www.openkg.cn/COVID-19/character/resour...,相关疾病
4,http://www.openkg.cn/COVID-19/character/resour...,http://www.openkg.cn/COVID-19/character/resour...,爆发地点
...,...,...,...
1155,http://www.openkg.cn/COVID-19/character/resour...,http://www.openkg.cn/COVID-19/character/resour...,发表
1156,http://www.openkg.cn/COVID-19/character/resour...,http://www.openkg.cn/COVID-19/character/resour...,发表
1157,http://www.openkg.cn/COVID-19/character/resour...,http://www.openkg.cn/COVID-19/character/resour...,发表
1158,http://www.openkg.cn/COVID-19/character/resour...,http://www.openkg.cn/COVID-19/character/resour...,发表


In [53]:
entity_df

,:ID,name,:LABEL
0,http://www.openkg.cn/COVID-19/character/resour...,马洪明,人物
1,http://www.openkg.cn/COVID-19/character/resour...,孙宝清,人物
2,http://www.openkg.cn/COVID-19/character/resour...,欧阳铭,人物
3,http://www.openkg.cn/COVID-19/character/resour...,刘升明,人物
4,http://www.openkg.cn/COVID-19/character/resour...,王大礼,人物
...,...,...,...
742,http://www.openkg.cn/COVID-19/character/resour...,《 病毒学报. 》,期刊
743,http://www.openkg.cn/COVID-19/character/resour...,《中国发明与专利》,期刊
744,http://www.openkg.cn/COVID-19/character/resour...,《生命科学》,期刊
745,http://www.openkg.cn/COVID-19/character/resour...,《 中国工程科学》,期刊


In [54]:
property_dict

{'P1': '籍贯',
 'P10': '参与事件',
 'P11': '发表',
 'P12': '事迹',
 'P13': '职责',
 'P14': '死亡原因',
 'P15': '出生地',
 'P16': '履历',
 'P18': '重要成果',
 'P19': '相关疾病',
 'P2': '性别',
 'P20': '导致',
 'P21': '顺承',
 'P22': '因果',
 'P23': '爆发地点',
 'P24': '所在城市',
 'P25': '颁布',
 'P26': '出版',
 'P28': '奖项名次',
 'P29': '获奖原因',
 'P3': '死亡时间',
 'P30': '发表日期',
 'P31': '结论',
 'P32': '发表',
 'P33': '第一作者',
 'P34': '共同作者',
 'P35': '发生地点',
 'P36': '职务',
 'P37': '任职机构',
 'P38': '所在城市',
 'P39': '获奖事件',
 'P4': '出生日期',
 'P40': '相关奖项',
 'P41': '出版日期',
 'P42': '同事',
 'P43': '合作',
 'P44': '队友',
 'P45': '支援地点',
 'P46': '所属救援队',
 'P48': '现任机构',
 'P49': '出发日期',
 'P5': '民族',
 'P6': '国籍',
 'P7': '研究方向',
 'P8': '毕业院校',
 'P9': '参与战役'}

In [55]:
nodes = get_nodes(entity_df, property_df)

In [56]:
property_df

,ID,Property,value
0,http://www.openkg.cn/COVID-19/character/resour...,奖项名次,二等奖
1,http://www.openkg.cn/COVID-19/character/resour...,奖项名次,一等奖
2,http://www.openkg.cn/COVID-19/character/resour...,奖项名次,二等奖
3,http://www.openkg.cn/COVID-19/character/resour...,奖项名次,一等奖
4,http://www.openkg.cn/COVID-19/character/resour...,籍贯,吉林省
...,...,...,...
284,http://www.openkg.cn/COVID-19/character/resour...,出生日期,1977-09-01
285,http://www.openkg.cn/COVID-19/character/resour...,民族,汉
286,http://www.openkg.cn/COVID-19/character/resour...,发表日期,2017-10-26
287,http://www.openkg.cn/COVID-19/character/resour...,发表日期,2019-05-26


In [57]:
nodes

,name,:LABEL,奖项名次,籍贯,性别,死亡时间,民族,出生日期,出版日期,发表日期,出发日期
:ID,,,,,,,,,,,
http://www.openkg.cn/COVID-19/character/resource/R100,马洪明,人物,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://www.openkg.cn/COVID-19/character/resource/R101,孙宝清,人物,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://www.openkg.cn/COVID-19/character/resource/R102,欧阳铭,人物,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://www.openkg.cn/COVID-19/character/resource/R103,刘升明,人物,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://www.openkg.cn/COVID-19/character/resource/R104,王大礼,人物,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
http://www.openkg.cn/COVID-19/character/resource/R95,《 病毒学报. 》,期刊,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://www.openkg.cn/COVID-19/character/resource/R96,《中国发明与专利》,期刊,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://www.openkg.cn/COVID-19/character/resource/R97,《生命科学》,期刊,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
nodes.to_csv('node.csv',encoding='utf-8')

In [30]:
relationship_df.columns=[':START_ID', ':END_ID', ':TYPE']
relationship_df.to_csv('relationship.csv',encoding='utf-8')

In [78]:
entity_df,property_dict = get_entity('medical-covid-19-v0.2.json')
#entity_df_id = entity_df.set_index(":ID")
relationship_df,property_df = get_rp('medical-covid-19-v0.2.json',property_dict)     

{'@graph': [{'@id': 'http://www.openkg.cn/COVID-19/medical/class/C1', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '病毒'}}, {'@id': 'http://www.openkg.cn/COVID-19/medical/class/C10', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '公告'}}, {'@id': 'http://www.openkg.cn/COVID-19/medical/class/C11', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '新闻'}}, {'@id': 'http://www.openkg.cn/COVID-19/medical/class/C12', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '期刊'}}, {'@id': 'http://www.openkg.cn/COVID-19/medical/class/C13', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '人物'}}, {'@id': 'http://www.openkg.cn/COVID-19/medical/class/C14', '@type': 'http://www.w3.org/2000/01/rdf-schema#Class', 'label': {'@language': 'zh', '@value': '研究领域'}}, {'@id': 'http://www.o

In [80]:
property_df

,ID,Property,value
0,http://www.openkg.cn/COVID-19/medical/resource...,使用建议,气虚、津伤血枯者孕妇服。
1,http://www.openkg.cn/COVID-19/medical/resource...,用量,煎服，3-10g。或入丸散。
2,http://www.openkg.cn/COVID-19/medical/resource...,临床应用,1.湿阻中焦，脘腹胀满。本品苦燥辛散，能燥湿，又下气除胀满，为消除胀满的要药。常与苍术、陈皮...
3,http://www.openkg.cn/COVID-19/medical/resource...,使用建议,阴虚血少者禁服。
4,http://www.openkg.cn/COVID-19/medical/resource...,用量,煎服，3-6g。
...,...,...,...
85,http://www.openkg.cn/COVID-19/medical/resource...,用量,煎服，5-10g。
86,http://www.openkg.cn/COVID-19/medical/resource...,临床应用,1.用于湿阻脾胃、脘腹胀满，寒湿白带，湿温病以及湿热下注、脚膝肿痛、痿软无力等症。 苍朮温...
87,http://www.openkg.cn/COVID-19/medical/resource...,使用建议,气虚证、阴虚燥咳、吐血证及舌赤少津、内有实热者慎服。
88,http://www.openkg.cn/COVID-19/medical/resource...,用量,内服：煎汤，3-10g；或入丸、散。


In [81]:
nodes = get_nodes(entity_df, property_df)

In [83]:
nodes.to_csv('medical_node.csv',encoding='utf-8')
relationship_df.columns=[':START_ID', ':END_ID', ':TYPE']
relationship_df.to_csv('medical_relationship.csv',encoding='utf-8')

In [17]:
property_dict

{'P1': '基本传染指数',
 'P10': '主要传播途径',
 'P100': '禁忌',
 'P101': '推荐中成药',
 'P102': '适用症',
 'P11': '易感染人群',
 'P12': '传染源',
 'P13': '属于',
 'P14': '引发',
 'P15': '研究进展',
 'P16': '并发症',
 'P17': '首次爆发疫区',
 'P18': '病例',
 'P19': '研究进展',
 'P2': '潜伏期',
 'P20': '防护手段',
 'P21': '症状',
 'P24': '指标',
 'P25': '伴随症状',
 'P26': '防护工具',
 'P27': '适用人群',
 'P28': '诊断标准',
 'P29': '判断依据',
 'P3': '结构',
 'P30': '临床分型',
 'P31': '证型',
 'P33': '发布',
 'P34': '相关',
 'P35': '所属期刊',
 'P36': '摘要',
 'P37': '类型',
 'P38': '文献',
 'P39': '表现',
 'P40': '推荐处方',
 'P41': '临床表现',
 'P42': '治疗方案',
 'P43': '发布时间',
 'P44': '简介',
 'P45': '内容',
 'P46': '发布时间',
 'P47': '简介',
 'P48': '内容',
 'P49': '治疗设备',
 'P5': '生理机能',
 'P50': '治疗药物',
 'P51': '属于',
 'P52': '测量设备',
 'P53': '检查项',
 'P54': '属于',
 'P55': '研究领域',
 'P56': '任职',
 'P57': '发表',
 'P58': '检查方式',
 'P59': '发布',
 'P6': '胸部影像学',
 'P60': '所属科目',
 'P61': '所属科目',
 'P62': '忌同食',
 'P63': '所含成分',
 'P66': '用量',
 'P67': '组成',
 'P68': '主治',
 'P69': '忌同食',
 'P7': '实验室检查',
 'P72': '功效',
 'P73': '归经',


In [13]:
nodes = get_nodes(entity_df, property_df)

NameError: name 'property_df' is not defined